In [2]:
!pip install -q selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/usr/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 5.3 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 http://security

In [3]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import pandas as pd

In [4]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [5]:
driver = webdriver.Chrome('chromedriver', options=options)
url = 'https://www.webscraper.io/test-sites/e-commerce/static/computers/laptops'
driver.get(url)

In [6]:
def phone_dataframe(url):    
    driver = webdriver.Chrome('chromedriver', options=options)
    driver.get(url)

    # Click the cookie pop-up so we don't raise a ElementClickInterceptedException
    driver.find_element(By.CLASS_NAME, 'acceptContainer').click()

    # Create a list to hold collected product titles
    data_list = []
    page_list = []
    pages_total = driver.find_elements(By.CLASS_NAME, "page-link")
    end = pages_total[len(pages_total) - 2].text
    end = int(end)

    # Set a pages left parameter that we deduct every time we scrape a full page
    pages_left = end

    # Condition of the loop proceeding while the pages_left parameter isn't 0
    while pages_left: 

        # Collect the price, title, description, and reviews on the first page 
        for vals in driver.find_elements(By.XPATH, "//div[@class='thumbnail']"):
            vals = re.split(r'\n', vals.text)
            data_list.append(vals)
        
        # Click to the next page if we're not on the last page 
        if pages_left != 1: 
            driver.find_element(By.XPATH, "//a[@rel='next']").click()
        
        # Explicitly wait for the page navigation buttons to be clickable again
        # following our previous click 
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH , "//a[@rel='prev']")))

        # Deduct our pages left stopping condition by 1 at the end of the loop
        pages_left -= 1

    # Once we've broken out of our while loop, transform our data storage list
    # into a pandas data frame 
    phone_df = pd.DataFrame(data_list)

    # Set our data column labels 
    phone_df.columns = ['Price', 'Title', 'Description', 'Reviews']

    # Return our data frame 
    return phone_df

In [7]:
phone_df = phone_dataframe("https://www.webscraper.io/test-sites/e-commerce/static/computers/laptops")
phone_df

,Price,Title,Description,Reviews
0,$416.99,Packard 255 G2,"15.6"", AMD E2-3800 1.3GHz, 4GB, 500GB, Windows...",2 reviews
1,$306.99,Aspire E1-510,"15.6"", Pentium N3520 2.16GHz, 4GB, 500GB, Linux",2 reviews
2,$1178.99,ThinkPad T540p,"15.6"", Core i5-4200M, 4GB, 500GB, Win7 Pro 64bit",2 reviews
3,$739.99,ProBook,"14"", Core i5 2.6GHz, 4GB, 500GB, Win7 Pro 64bit",8 reviews
4,$1311.99,ThinkPad X240,"12.5"", Core i5-4300U, 8GB, 240GB SSD, Win7 Pro...",12 reviews
...,...,...,...,...
112,$1349.23,Lenovo ThinkPad...,"Lenovo ThinkPad T470, 14"" FHD IPS, Core i5-720...",5 reviews
113,$1362.24,Lenovo ThinkPad...,"Lenovo ThinkPad Yoga 370 Black, 13.3"" FHD IPS ...",12 reviews
114,$1366.32,Toshiba Portege...,"Toshiba Portege X20W-D-10V Black/Blue, 12.5"" F...",11 reviews
115,$1381.13,Asus ASUSPRO B94...,"Asus ASUSPRO B9440UA-GV0279R Gray, 14"" FHD, Co...",4 reviews
